In [1]:
import numpy as np
import random
import pandas as pd 
from copy import deepcopy
import math
import time

In [2]:
def rastrigin(x):
    A = 10
    n = len(x)
    return -(A*n + sum(x[i]**2 - A*np.cos(2*np.pi*x[i]) for i in range(n)))

In [3]:
def rosenbrock(x):
    a = 1
    b = 100
    return -((a - x[0])**2 + b*(x[1] - x[0]**2)**2)

In [4]:
def shaffer(x):
    return -( 0.5 + (np.sin((x[0]**2 +x[1]**2))**2-0.5)/(1.0 + 0.001*(x[0]**2+x[1]**2))**2)

In [5]:
class Individual:
    global_best_position=None
    global_best_fitness=None
    def __init__(self,bounds, velocity_calc,calc_fitness):
        self.lower_bounds = np.array([bound[0] for bound in bounds])
        self.upper_bounds = np.array([bound[1] for bound in bounds])        
        
        self.position = np.random.uniform(self.lower_bounds, self.upper_bounds, len(bounds))
        self.velocity = np.random.uniform(self.lower_bounds - self.upper_bounds,
                                          self.upper_bounds - self.lower_bounds,
                                          len(bounds))
        self.calc_fitness=calc_fitness
        self.fitness=calc_fitness(self.position)
        

        self.velocity_calc=velocity_calc
        
        self.personal_best_fitness=self.fitness
        self.personal_best_position=self.position.copy()
        
        if Individual.global_best_fitness is None or Individual.global_best_fitness<self.fitness:
                Individual.global_best_fitness=self.fitness
                Individual.global_best_position=self.position.copy()
    
        
    def update_position(self):
        self.position = np.clip(self.position + self.velocity,
                                self.lower_bounds,
                                self.upper_bounds)
        self.update_fitness()
       
                
    def update_velocity(self):
        personal_direction=(self.personal_best_position - self.position)
        global_direction=(Individual.global_best_position - self.position)
                
        cognitive_velocity =random.random() * self.velocity_calc["cognitive"]*personal_direction
        social_velocity =random.random() * self.velocity_calc["group"]* global_direction
        
        inertia = self.velocity_calc["inertia"] * self.velocity
        self.velocity = inertia + cognitive_velocity + social_velocity
        self.velocity=np.clip(self.velocity,2*self.lower_bounds,2*self.upper_bounds)
    def update_fitness(self):
        self.fitness=self.calc_fitness(self.position)
        if self.personal_best_fitness is None or self.fitness>self.personal_best_fitness:
            self.personal_best_fitness=self.fitness
            self.personal_best_position=self.position.copy()
            if Individual.global_best_fitness is None or Individual.global_best_fitness<self.fitness:
                Individual.global_best_fitness=self.fitness
                Individual.global_best_position=self.position.copy()
    

In [6]:
def selection(population:Individual) ->Individual:
    #roulete
    return random.choices(population=list(zip(range(len(population)),population)),weights=[-ind.fitness for ind in population],k=1)[0]

In [7]:
def crossover(p1:Individual,p2:Individual,c1:Individual,c2:Individual):
    coef=random.random()
    c1.position=coef*p1.position + (1-coef)*p2.position

    c2.position=coef*p2.position +(1-coef)*p1.position
    

In [8]:
def mutation(ind:Individual,mut_prob=0.01):
    #TODO
    pass

In [9]:
def GA(population:[Individual],pop_size,num_iters:int,elitism_size=6):
    
    new_population=population[:]
    for _ in range(0,num_iters):
        sorted(population,key=lambda x:x.fitness,reverse=True)
        for i in range(elitism_size,pop_size,2):
            p1=selection(population)[1]
            p2=selection(population)[1]
            
            crossover(p1,p2,new_population[i],new_population[i+1])
            mutation(new_population[i])
            mutation(new_population[i+1])
        population=new_population[:]
    

In [10]:
vezbe_velocity_calc={
    "cognitive":0.7 ,
    "group":1.0 ,
    "inertia":1.0
}

In [11]:
def hybrid(swarm_size,num_dimensions,num_iters,allowed_time,velocity_calc,fitness_calc):
    
    bounds = [(-5.12, 5.12) for _ in range(num_dimensions)]
    
    swarm = [Individual(bounds=bounds,
                      calc_fitness=fitness_calc,velocity_calc=velocity_calc,
                      ) for i in range(4*swarm_size)]
    
    
    
    start=time.time()
    GA(swarm,swarm_size*4,int(num_iters/4))
    sorted(swarm,key=lambda x:x.fitness,reverse=True)
    swarm=swarm[:swarm_size]
    for psoiter in range(num_iters):
        
        for i in range(len(swarm)):
               
            swarm[i].update_velocity()
            swarm[i].update_position()
            
            swarm[i].update_fitness()
       
        
        if time.time()-start>allowed_time:
                tmp=Individual.global_best_fitness
                Individual.global_best_fitness=None
                Individual.global_best_position=None
                return tmp
    print("got out")
    return Individual.global_best_fitness

In [12]:
hybrid(swarm_size=50,num_dimensions=2,num_iters=700,allowed_time=1,velocity_calc=vezbe_velocity_calc,fitness_calc=rosenbrock)

got out


-5.910146291210176e-05

In [13]:
hybrid(swarm_size=50,num_dimensions=2,num_iters=800,allowed_time=1,velocity_calc=vezbe_velocity_calc,fitness_calc=rastrigin)

-5.910146291210176e-05

In [14]:
hybrid(swarm_size=50,num_dimensions=2,num_iters=900,allowed_time=1,velocity_calc=vezbe_velocity_calc,fitness_calc=shaffer)

-1.0346047907283662e-09